# Spectral Clustering

Now we will explore the ins and outs of spectral clustering for graph and other data. Spectral clustering is a technique with roots in graph theory, where the approach is used to identifyu commnunities of nodes in graph based on the edges connecting them. This method isn't exclusive to cluster, it can work with other kinds of data. 

Spectral clustering uses information from the eigenvalues (spectrum) of special matrices built from the graph or the dataset.

## Eigenvectors and eigenvalues

We can think of a matrix A as a function which map vector to new vectors. Most vectors will end up somewhere completly different when A is applied to them, but eigenvectors only changes in magnitude. If you drew a line through the origin and the eigenvector, then after the mapping, the eigenvector would still land on the line. The amount which the vector is scaled along the line depends on $\lambda$ (eigenvalue).

Eigenvectors are an important part of linear algebra, because they help describe the dynamics of systems represented by matrices

In [2]:
import numpy as np

A = np.array([[0,1],[-2,-3]])

vals, vecs = np.linalg.eig(A)

for i, value in enumerate(vals):
    print("Eigenvector:", vecs[:,1],"Eigenvalue:", value)

Eigenvector: [-0.4472136   0.89442719] Eigenvalue: -1.0
Eigenvector: [-0.4472136   0.89442719] Eigenvalue: -2.0


## Graphs

Graphs are natural way to reprsent many types of data. A graph is a set of nodes with a corresponding set of edges which connect the nodes. The edges may be directed or undirected and can even have weights associated with them. 

## Degree Matrix

The degree of a node is how many edges connect to it. In a directed graph we could talk about in-degree and out-degree. The degree matrix is a diagonal matrix where the value at entry (i,i) is the degree of node i. 

In [3]:
A = np.array([
  [0, 1, 1, 0, 0, 0, 0, 0, 1, 1],
  [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
  [1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
  [0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
  [0, 0, 0, 1, 1, 0, 1, 1, 0, 0],
  [0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
  [0, 0, 0, 0, 0, 1, 1, 0, 0, 0],
  [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
  [1, 0, 0, 0, 0, 0, 0, 0, 1, 0]])

D = np.diag(A.sum(axis=1))
print(D)

[[4 0 0 0 0 0 0 0 0 0]
 [0 2 0 0 0 0 0 0 0 0]
 [0 0 2 0 0 0 0 0 0 0]
 [0 0 0 2 0 0 0 0 0 0]
 [0 0 0 0 2 0 0 0 0 0]
 [0 0 0 0 0 4 0 0 0 0]
 [0 0 0 0 0 0 2 0 0 0]
 [0 0 0 0 0 0 0 2 0 0]
 [0 0 0 0 0 0 0 0 2 0]
 [0 0 0 0 0 0 0 0 0 2]]


## Graph Laplacian

The laplacian is just another matrix representation of a graph. It has several beautiful properties, which we will take advange of for spectral clustering. In fact, the number of 0 eigenvalues corresponds to the number of connected components in our graph! To calculate the normal Laplacian (there are several variants), we just subtract the adjacency matrix from our degree matrix:

In [4]:
L = D - A
print(L)

[[ 4 -1 -1  0  0  0  0  0 -1 -1]
 [-1  2 -1  0  0  0  0  0  0  0]
 [-1 -1  2  0  0  0  0  0  0  0]
 [ 0  0  0  2 -1 -1  0  0  0  0]
 [ 0  0  0 -1  2 -1  0  0  0  0]
 [ 0  0  0 -1 -1  4 -1 -1  0  0]
 [ 0  0  0  0  0 -1  2 -1  0  0]
 [ 0  0  0  0  0 -1 -1  2  0  0]
 [-1  0  0  0  0  0  0  0  2 -1]
 [-1  0  0  0  0  0  0  0 -1  2]]


The first nonzero eigenvalue is called the spectral gap. The spectral gap gives us some notion of the density of the graph. If this graph was densely connected (all pairs of the 10 nodes had an edge), then the spectral gap would be 10.

The second eigenvalue is called the Fiedler value, and the corresponding vector is the Fiedler vector. The Fiedler value approximates the minimum graph cut needed to separate the graph into two connected components. Recall, that if our graph was already two connected components, then the Fiedler value would be 0. Each value in the Fiedler vector gives us information about which side of the cut that node belongs. Let’s color the nodes based on whether their entry in the Fielder vector is positive or not:

This simple trick has segmented our graph into two clusters! Why does this work? Remember that zero eigenvalues represent connected components. Eigenvalues near zero are telling us that there is almost a separation of two components. Here we have a single edge, that if it didn’t exist, we’d have two separate components. So the second eigenvalue is small.

To summarize what we know so far: the first eigenvalue is 0 because we have one connected component. The second eigenvalue is near 0 because we’re one edge away from having two connected components. We also saw that the vector associated with that value tells us how to separate the nodes into those approximately connected components.

You may have noticed that the next two eigenvalues are also pretty small. That tells us that we are “close” to having four separate connected components. In general, we often look for the first large gap between eigenvalues in order to find the number of clusters expressed in our data. See the gap between eigenvalues four and five?

Having four eigenvalues before the gap indicates that there is likely four clusters. The vectors associated with the first three positive eigenvalues should give us information about which three cuts need to be made in the graph to assign each node to one of the four approximated components. Let’s build a matrix from these three vectors and perform K-Means clustering to determine the assignments:

In [5]:
from sklearn.cluster import KMeans

# our adjacency matrix
print("Adjacency Matrix:")
print(A)

# diagonal matrix
D = np.diag(A.sum(axis=1))

# graph laplacian
L = D-A

# eigenvalues and eigenvectors
vals, vecs = np.linalg.eig(L)

# sort these based on the eigenvalues
vecs = vecs[:,np.argsort(vals)]
vals = vals[np.argsort(vals)]

# kmeans on first three vectors with nonzero eigenvalues
kmeans = KMeans(n_clusters=4)
kmeans.fit(vecs[:,1:4])
colors = kmeans.labels_

print("Clusters:", colors)

Adjacency Matrix:
[[0 1 1 0 0 0 0 0 1 1]
 [1 0 1 0 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 1 0 0 0 0]
 [0 0 0 1 0 1 0 0 0 0]
 [0 0 0 1 1 0 1 1 0 0]
 [0 0 0 0 0 1 0 1 0 0]
 [0 0 0 0 0 1 1 0 0 0]
 [1 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 1 0]]
Clusters: [2 1 1 3 3 0 0 0 2 2]


## Spectral Clustering Arbitrary Data 

Obviously, K-Means wasn’t going to work. It operates on euclidean distance, and it assumes that the clusters are roughly spherical. This data (and often real world data) breaks these assumptions. Let’s try to tackle this with spectral clustering.

### Nearest Neighbors Graph

There are a couple of ways to treat our data as a graph. The easiest way is to construct a k-nearest neighbors graph. A k-nearest neighbors graph treats every data point as a node in a graph. An edge is then drawn from each node to its k nearest neighbors in the original space. Generally, the algorithm isn’t too sensitive of the choice of k. Smaller numbers like 5 or 10 usually work pretty well.

Look at the picture of the data again, and imagine that each point is connected to its 5 closest neighbors. Any point in the outer ring should be able to follow a path along the ring, but there won’t be any paths into the inner circle. It’s pretty easy to see that this graph will have two connected components: the outer ring and the inner circle.

Since we’re only separating this data into two components, we should be able to use our Fiedler vector trick from before. Here’s the code I used to perform spectral clustering on this data:

In [6]:
from sklearn.datasets import make_circles
from sklearn.neighbors import kneighbors_graph
import numpy as np

# create the data
X, labels = make_circles(n_samples=500, noise=0.1, factor=.2)

# use the nearest neighbor graph as our adjacency matrix
A = kneighbors_graph(X, n_neighbors=5).toarray()
print(A)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [7]:
# create the graph laplacian
D = np.diag(A.sum(axis=1))
L = D-A

# find the eigenvalues and eigenvectors
vals, vecs = np.linalg.eig(L)

# sort
vecs = vecs[:,np.argsort(vals)]
vals = vals[np.argsort(vals)]

# use Fiedler value to find best cut to separate data
clusters = vecs[:,1] > 0

## Other Approaches

The nearest neighbor graph is a nice approach, but it relies on the fact that “close” points should belong in the same cluster. Depending on your data, that might not be true. A more general approach is to construct an affinity matrix. An affinity matrix is just like an adjacency matrix, except the value for a pair of points expresses how similar those points are to each other. If pairs of points are very dissimilar then the affinity should be 0. If the points are identical, then the affinity might be 1. In this way, the affinity acts like the weights for the edges on our graph.

How to decide on what it means for two data points to be similar is one of the most important questions in machine learning. Often domain knowledge is the best way to construct a similarity measure. If you have access to domain experts, ask them this question.

There are also entire fields dedicated to learning how to construct similarity metrics directly from data. For example, if you have some labeled data, you can train a classifier to predict whether two inputs are similar or not based on if they have the same label. This classifier can then be used to assign affinity to pairs of unlabeled points.

## Conclusion

We’ve covered the theory and application of spectral clustering for both graphs and arbitrary data. Spectral clustering is a flexible approach for finding clusters when your data doesn’t meet the requirements of other common algorithms.

First, we formed a graph between our data points. The edges of the graph capture the similarities between the points. The eigenvalues of the Graph Laplacian can then be used to find the best number of clusters, and the eigenvectors can be used to find the actual cluster labels.

I hope you enjoyed this post and find spectral clustering useful in your work or exploration.